## $S_8 - \Omega_m$ and $\sigma_8-\Omega_m$ Contour Plot

This notebook creates a 2D marginalised contour plot of the $S_8$ and $\Omega_m$ parameters, comparing this work with fiducial results from DES Y3, HSC and KiDS-Legacy (no chains yet so maybe just KiDS-1000?). Also the same for a plot of $\sigma_8-\Omega_m$.

This notebook also reds the chains and derives the posterior values of $S_8$, subsequently writing them intoa file to be read in and plotted as a whisker plot in S8_whisker.ipynb.

In [13]:
from getdist import plots, loadMCSamples
import numpy as np
import matplotlib.pyplot as plt
import uncertainties
import os

plt.rcParams.update({"text.usetex": True})

g = plots.get_subplot_plotter(width_inch=12)
g.settings.axes_fontsize=30
g.settings.axes_labelsize=40
g.settings.alpha_filled_add = 0.7
g.settings.legend_fontsize = 20

%matplotlib inline

#SPECIFY DATA DIRECTORY AND DESIRED CHAINS TO ANALYSE
root_dir='/n23data1/n06data/lgoh/scratch/UNIONS/chains/ext_data'
roots = [
    'DES_Y3',
    'HSC_Y3',
    'DES+KiDS',
    'KiDS-1000',
    'Planck18'
    ]

root_label = [
    r'DES Y3 ($\xi_\pm$)',
    r'HSC Y3 ($\xi_\pm$)',
    r'DES+KiDS ($\xi_\pm$)',
    r'KiDS-1000 ($\xi_\pm$)',
    r'Planck18 TT+TE+EE+lowE'
    ]


In [2]:
# Create the .paramnames file needed by getdist (take the parameter names from the first line of the output file)
for root in roots:
    if os.path.isfile(f"{root_dir}/{root}/getdist_{root}.paramnames")==False:
        
        with open(f"{root_dir}/{root}/{root}.txt", "r") as file:
            params = file.readline()[1:].split('\t')[:-4]
            file.close()
    
        with open(f"{root_dir}/{root}/getdist_{root}.paramnames", "w") as file:
            for i in range(len(params)):
                if len(params[i].split('--')) > 1:
                    file.write(params[i].split('--')[1] + '\n')
                else:
                    file.write(params[i].split('--')[0] + '\n')
            file.close()
        print(params)

## Retrieve the chains

In [14]:
#READ CHAIN

chains=[]

for root in roots:
    if os.path.isfile(f"{root_dir}/{root}/getdist_{root}.paramnames")==False:
        samples = np.loadtxt(f"{root_dir}/{root}/{root}.txt")
        samples = np.column_stack((samples[:,-1],samples[:,-3],samples[:,0:-4]))
        np.savetxt(f"{root_dir}/{root}/getdist_{root}.txt", samples)
    
    chain = g.samples_for_root(f"{root_dir}/{root}/getdist_{root}",
                               cache=False,
                               settings={'ignore_rows':0,
                                        #  'smooth_scale_2D':0.7,
                                        #  'smooth_scale_1D':0.7
                                         })
    p = chain.getParams()
    chain.addDerived(p.omega_m, name='OMEGA_M', label=r'$\Omega_m$')
    chain.addDerived(p.SIGMA_8, name='sigma_8', label=r'$\sigma_8$')
    if hasattr(p,'S_8')==False:
        chain.addDerived(p.SIGMA_8*(np.sqrt(p.omega_m/0.3)), name='S_8', label=r'S_8')

    chains.append(chain)

## Plot the chain

In [ ]:
%matplotlib inline
g.plot_2d(chains,
        ['OMEGA_M','S_8'],
        # contour_colors=['#0072B2', '#009E73', '#D55E00', '#CC79A7'],
        filled=True)
g.add_legend(root_label, legend_loc='upper left')
# g.export('contour_plot_s8_om_fid.pdf')

In [ ]:
%matplotlib inline
g.plot_2d(chains,
        ['OMEGA_M','sigma_8'],
        # contour_colors=['#0072B2', '#009E73', '#D55E00', '#CC79A7'],
        filled=True)
g.add_legend(root_label, legend_loc='upper left')
# g.export('contour_plot_sigma8_om_fid.pdf')

### Output bestfit and sigma values of parameters

In [ ]:
#########BESTFIT AND SIGMA VALS##########
params = ['OMEGA_M','S_8']
latex_params = [r'$\Omega_{m}$',r'$S_8$']

chains = chains[::-1]
for chain in chains:

    margestats = chain.getMargeStats()
    likestats = chain.getLikeStats()
    p=chain.getParams()

    for no in range(len(latex_params)):
        if hasattr(p,params[no]):
            param_stats = margestats.parWithName(params[no])
            a = np.array([param_stats.mean,param_stats.mean-param_stats.limits[0].lower, param_stats.limits[0].upper-param_stats.mean])
            if '%.2g' %a[1] == '%.2g' %a[2]:
                latex_params[no] += '&$%.3g\pm%.2g$'%(a[0],a[1])
            else:
                latex_params[no] += '&$%.3g_{-%.2g}^{+%.2g}$'%(a[0],a[1],a[2])
        else:
            latex_params[no] += '&$-$'

        
for param in latex_params:
    param += r'\\'
    print(param)   


### Save values of $S_8$

In [ ]:
s8_values = np.array(["# Expt", "Mean", "S8_low", "S8_high"])
for i, chain in enumerate(chains):

    margestats = chain.getMargeStats()
    likestats = chain.getLikeStats()

    param_stats = margestats.parWithName('S_8')
    
    s8_values = np.vstack((s8_values,[root_label[i], param_stats.mean, param_stats.mean-param_stats.limits[0].lower, param_stats.limits[0].upper-param_stats.mean]))
print(s8_values)
np.savetxt(f"{root_dir}/S8_means.txt", s8_values, fmt=['%s','%s','%s','%s'], delimiter=',')